# Imports,  Settings and Paths

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd#
%matplotlib inline
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import time

In [2]:
from Code.Models.c7o2h10_model import DeepPotential, train, normalize, backtransform

In [3]:
data_path = './Dataset/c7o2h10_X.npy'
label_path = './Dataset/c7o2h10_Y.npy'
train_ids_path = './Dataset/iso17/train_ids.txt'
test_ids_path = './Dataset/iso17/validation_ids.txt'
model_path = './ModelCheckpoints/c7o2h10/'

In [4]:
train_ids = (np.loadtxt(train_ids_path) - 1).astype(int).tolist()
test_ids = (np.loadtxt(test_ids_path) - 1).astype(int).tolist()

In [5]:
use_cuda = False#torch.cuda.is_available()
print('use cuda:  ', use_cuda)

use cuda:   False


# Read the Data

In [6]:
X_npy = np.load(data_path)
Y_npy = np.load(label_path) * -1
Y_npy, Y_min, Y_max = normalize(Y_npy)
X_npy.shape

(404000, 19, 72)

### Check for NaN values

In [7]:
use_ids = np.where(np.isnan(X_npy).sum(axis=2).sum(axis=1)==0)
print('{} Datapoints with NaN values found.'.format(X_npy.shape[0] - len(use_ids[0])))

0 Datapoints with NaN values found.


In [8]:
X_npy = X_npy[use_ids]
Y_npy = Y_npy[use_ids]
X_npy.shape

(404000, 19, 72)

In [9]:
X_npy = X_npy
Y_npy = Y_npy

### Split into test and train

In [10]:
X_train_npy, X_test_npy = X_npy[train_ids], X_npy[test_ids]
Y_train_npy, Y_test_npy = Y_npy[train_ids], Y_npy[test_ids]
print('X Train shape: {}\tX Test shape: {}\nY Train shape: {}\tY Test shape: {}'.format(X_train_npy.shape,
                                                                                        X_test_npy.shape,
                                                                                        Y_train_npy.shape,
                                                                                        Y_test_npy.shape))

X Train shape: (400000, 19, 72)	X Test shape: (4000, 19, 72)
Y Train shape: (400000,)	Y Test shape: (4000,)


### Shuffle the Dataset

In [11]:
shuffle = np.arange(X_train_npy.shape[0])
np.random.shuffle(shuffle)
X_train_npy = X_train_npy[shuffle]
Y_train_npy = Y_train_npy[shuffle]

In [19]:
X_train_npy[0]

array([[ 1.99828394e-01,  2.50727784e-01,  9.98340126e-01, ...,
         5.46648651e-02,  9.38760591e-01,  3.44570098e-01],
       [ 2.71495815e-01, -3.40650009e-01, -9.96749098e-01, ...,
         6.12323400e-17,  1.00000000e+00,  0.00000000e+00],
       [ 2.46016518e-01, -1.31295310e-01,  6.74797711e-01, ...,
        -7.88248014e-01,  7.21116713e-01,  6.92813602e-01],
       ...,
       [ 2.31034241e-01,  5.09698171e-02,  8.34332155e-01, ...,
         2.79976721e-01,  3.24676124e-01,  9.45825256e-01],
       [ 2.10615484e-01,  5.34513144e-01,  8.70427767e-01, ...,
         6.12323400e-17,  6.78813494e-01,  7.34310725e-01],
       [ 2.03229327e-01, -9.77240134e-02,  9.86554002e-01, ...,
         6.12323400e-17,  7.35176631e-01,  6.77875594e-01]])

In [20]:
X_train_npy[shuffle][0]

array([[ 2.04567829e-01,  1.25079741e-01,  9.73910254e-01, ...,
         6.58605407e-01,  9.98071014e-01,  6.20826143e-02],
       [ 2.92962433e-01, -5.30651518e-01,  9.99017877e-01, ...,
         4.25954523e-01,  9.96313315e-01, -8.57891504e-02],
       [ 2.54718642e-01, -6.72839207e-01,  3.41865795e-01, ...,
        -6.72002621e-01,  4.30951319e-02,  9.99070973e-01],
       ...,
       [ 1.98928878e-01, -3.50035251e-01,  7.40095612e-01, ...,
         6.12323400e-17,  1.00000000e+00,  0.00000000e+00],
       [ 2.17100268e-01, -5.43929750e-02,  7.79389337e-01, ...,
        -3.40561373e-01,  5.75153817e-01,  8.18045284e-01],
       [ 1.89513885e-01, -1.48128974e-01,  9.19630197e-01, ...,
         2.87413527e-01,  5.25287391e-01,  8.50924883e-01]])

### Create Pytorch Variables

In [13]:
if use_cuda:
    X_train = Variable(torch.Tensor(X_train_npy).cuda())
    Y_train = Variable(torch.Tensor(Y_train_npy).cuda(), requires_grad=False)
    X_test = Variable(torch.Tensor(X_test_npy).cuda())
    Y_test = Variable(torch.Tensor(Y_test_npy).cuda(), requires_grad=False)
else:
    X_train = Variable(torch.Tensor(X_train_npy))
    Y_train = Variable(torch.Tensor(Y_train_npy), requires_grad=False)
    X_test = Variable(torch.Tensor(X_test_npy))
    Y_test = Variable(torch.Tensor(Y_test_npy), requires_grad=False)

# Train Model

## Do the learning

In [14]:
if use_cuda:
    init_deep_pot = DeepPotential().cuda()
else:
    init_deep_pot = DeepPotential()
init_optim = torch.optim.Adam(init_deep_pot.parameters())
deep_pot, optim = train(init_deep_pot, init_optim, X_train, Y_train,
                        X_test, Y_test,
                        300, 128, (0.01, 0.96, 1.5),
                        checkpoint_path=model_path, print_every=100)


Epoch: 0	learning rate: 0.01
---


KeyboardInterrupt: 

In [ ]:
#deep_pot = DeepPotential().cuda()
#deep_pot.load_state_dict(torch.load('./ModelCheckpoints/epoch_13'))
Y_result = deep_pot.forward(X_test)

In [ ]:
result = backtransform(np.array(Y_result.data.tolist()), Y_min, Y_max)

In [ ]:
test_labels = backtransform(np.array(Y_test), Y_min, Y_max)

In [ ]:
np.mean(np.abs(test_labels - result))

In [ ]:
test_labels[:10]

In [ ]:
result[:10]

## Load Model

In [ ]:
test_model = DeepPotential().cuda()
test_model.load_state_dict(torch.load('ModelCheckpoints/c7o2h10/epoch_280'))

In [ ]:
test_result = np.squeeze(backtransform(test_model(X_test), Y_min, Y_max).detach().cpu().numpy())

In [ ]:
val_result = np.squeeze(backtransform(Y_test, Y_min, Y_max).detach().cpu().numpy())

In [ ]:
np.mean(np.abs(test_result - val_result))

In [ ]:
val_result[:10]

## Test the model
### Mean Absolute Error
The desired accuracy is about 0.04 eV

In [ ]:
mae = np.abs(deep_pot.forward(X_data[-500:]).data.numpy().reshape(500) - Y_data[-500:].data.numpy().reshape(500)).mean()
print('The nural network reaches a mean absolute error of {} eV'.format(mae))

### Small test sample

In [ ]:
deep_pot.forward(X_data[-10:])

In [ ]:
Y_data[-10:]

## Save the Model Parameters

In [ ]:
torch.save(deep_pot.state_dict(), model_path)

## Continue learning

In [ ]:
deep_pot = DeepPotential()
deep_pot.load_state_dict(torch.load(model_path))
optim = torch.optim.Adam(deep_pot.parameters(), lr=1e-2)

In [ ]:
deep_pot, optim = train(deep_pot, optim, X_data, Y_data, 1000, 512, (0.001, 0.96, 1.5), use_for_train=0.9, print_every=10)